In [1]:
"""pydca demo

Author: Evan Cresswell-Clay
"""
import sys,os
import data_processing as dp
import ecc_tools as tools
import timeit
# import pydca-ER module
from pydca.erdca import erdca
from pydca.sequence_backmapper import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.msa_trimmer.msa_trimmer import MSATrimmerException
from pydca.dca_utilities import dca_utilities
import numpy as np
import pickle
from gen_ROC_jobID_df import add_ROC
import matplotlib.pyplot as plt

# Import Bio data processing features 
import Bio.PDB, warnings
from Bio.PDB import *
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning

warnings.filterwarnings("error")
warnings.simplefilter('ignore', BiopythonWarning)
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', ResourceWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")



pfam_id = 'PF02146'
pfam_id = 'PF00186'
pfam_id = 'PF17795'

protein_msa_file = 'orig_MSA_%s.fa'%pfam_id
protein_refseq_file = 'orig_ref_%s.fa'%pfam_id
#protein_msa_file = 'MSA_PF00186.fa'
#protein_refseq_file = 'ref_PF00186.fa'

## Use Polypeptide Sequence from PDB as your reference sequence in MSA.

In [2]:

data_path = '../../../Pfam-A.full'
data_path = '/data/cresswellclayec/hoangd2_data/Pfam-A.full'
data_path = '/home/eclay/Pfam-A.full'

# pdb_ref should give list of
# 0) accession codes,
# 1) number of residues,
# 2) number of sequences,
# 3) and number of pdb references
# Read in Reference Protein Structure
pdb = np.load('%s/%s/pdb_refs.npy'%(data_path,pfam_id))
# convert bytes to str (python 2 to python 3)                                                                       
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0])      for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])
ipdb = 0
tpdb = int(pdb[ipdb,1])
print('Ref Sequence # should be : ',tpdb-1)

# Load Multiple Sequence Alignment
s = dp.load_msa(data_path,pfam_id)

# Load Polypeptide Sequence from PDB as reference sequence
print(pdb[ipdb,:])
pdb_id = pdb[ipdb,5]
pdb_chain = pdb[ipdb,6]
pdb_start,pdb_end = int(pdb[ipdb,7]),int(pdb[ipdb,8])
#print('pdb id, chain, start, end, length:',pdb_id,pdb_chain,pdb_start,pdb_end,pdb_end-pdb_start+1)                        
pdb_range = [pdb_start, pdb_end]
#print('download pdb file')                                                                       
pdb_file = pdb_list.retrieve_pdb_file(str(pdb_id),file_format='pdb')
#pdb_file = pdb_list.retrieve_pdb_file(pdb_id)                                                    
#---------------------------------------------------------------------------------------------------------------------#            
chain = pdb_parser.get_structure(str(pdb_id),pdb_file)[0][pdb_chain]
ppb = PPBuilder().build_peptides(chain)
#    print(pp.get_sequence())
print('peptide build of chain produced %d elements'%(len(ppb)))

found_match = True
matching_seq_dict = {}
poly_seq = list()
for i,pp in enumerate(ppb):
    for char in str(pp.get_sequence()):
        poly_seq.append(char)    
print('PDB Polypeptide Sequence (len=%d): \n'%len(poly_seq),poly_seq)
poly_seq_range = poly_seq[pdb_range[0]:pdb_range[1]]
print('PDB Polypeptide Sequence (In Proteins PDB range len=%d): \n'%len(poly_seq_range),poly_seq_range)

#check that poly_seq matches up with given MSA
    
pp_msa_file, pp_ref_file = tools.write_FASTA(poly_seq_range, s, pfam_id, number_form=False,processed=False)

        



Ref Sequence # should be :  2
shape of s (import from msa.npy):
 (271, 139)
shape of s (after UTF-8 decode):
 (271, 139)
['PF17795' '3' 'MVP_RAT' '457' '518' '4HL8' 'A' '457' '518']
Structure exists: '/home/eclay/DCA_ER/biowulf/hl/pdb4hl8.ent' 
peptide build of chain produced 3 elements
PDB Polypeptide Sequence (len=782): 
 ['E', 'A', 'I', 'I', 'R', 'I', 'P', 'P', 'Y', 'H', 'Y', 'I', 'H', 'V', 'L', 'D', 'Q', 'N', 'S', 'N', 'V', 'S', 'R', 'V', 'E', 'V', 'G', 'P', 'K', 'T', 'Y', 'I', 'R', 'Q', 'D', 'N', 'E', 'R', 'V', 'L', 'F', 'A', 'P', 'V', 'R', 'M', 'V', 'T', 'V', 'P', 'P', 'R', 'H', 'Y', 'C', 'I', 'V', 'A', 'N', 'P', 'V', 'S', 'R', 'D', 'T', 'Q', 'S', 'S', 'V', 'L', 'F', 'D', 'I', 'T', 'G', 'Q', 'V', 'R', 'L', 'R', 'H', 'A', 'D', 'Q', 'E', 'I', 'R', 'L', 'A', 'Q', 'D', 'P', 'F', 'P', 'L', 'Y', 'P', 'G', 'E', 'V', 'L', 'E', 'K', 'D', 'I', 'T', 'P', 'L', 'Q', 'V', 'V', 'L', 'P', 'N', 'T', 'A', 'L', 'H', 'L', 'K', 'A', 'L', 'L', 'D', 'F', 'E', 'D', 'K', 'N', 'G', 'D', 'K', 'V', 'M', 'A'

In [3]:
#just add using muscle:
#https://www.drive5.com/muscle/manual/addtomsa.html
muscling = True
muscle_msa_file = 'PP_muscle_msa_'+pfam_id+'.fa'
if muscling:
        #just add using muscle:
        #https://www.drive5.com/muscle/manual/addtomsa.html
        #https://www.drive5.com/muscle/downloads.htmL
        os.system("./muscle -profile -in1 %s -in2 %s -out %s"%(pp_msa_file,pp_ref_file,muscle_msa_file))
        print("PP sequence added to alignment via MUSCLE")


PP sequence added to alignment via MUSCLE


In [4]:
preprocessing = True

preprocessed_data_outfile = 'MSA_%s_PreProcessed.fa'%pfam_id
if preprocessing:
        # create MSATrimmer instance 
        if muscling:
            trimmer = msa_trimmer.MSATrimmer(
                muscle_msa_file, biomolecule='PROTEIN',
                refseq_file=pp_ref_file
            )
        else:
            trimmer = msa_trimmer.MSATrimmer(
                pp_msa_file, biomolecule='PROTEIN',
                refseq_file=pp_ref_file
            )  

        # Adding the data_processing() curation from tools to erdca.
        try:
                preprocessed_data,s_index, cols_removed,s_ipdb,s = trimmer.get_preprocessed_msa(printing=True, saving = False)
        except(MSATrimmerException):
                ERR = 'PPseq-MSA'
                print('Error with MSA trimms (%s)'%ERR)
                sys.exit()

        # Save processed data dictionary and FASTA file
        pfam_dict = {}
        pfam_dict['s0'] = s
        pfam_dict['msa'] = preprocessed_data
        pfam_dict['s_index'] = s_index
        pfam_dict['s_ipdb'] = s_ipdb
        pfam_dict['cols_removed'] = cols_removed

        input_data_file = "pfam_ecc/%s_DP.pickle"%(pfam_id)
        with open(input_data_file,"wb") as f:
                pickle.dump(pfam_dict, f)
        f.close()

        #write trimmed msa to file in FASTA format
        with open(preprocessed_data_outfile, 'w') as fh:
            for seqid, seq in preprocessed_data:
                fh.write('>{}\n{}\n'.format(seqid, seq))
else:
        input_data_file = "pfam_ecc/%s_DP.pickle"%(pfam_id)
        with open(input_data_file,"rb") as f:
                pfam_dict =  pickle.load(f)
        f.close()
        cols_removed = pfam_dict['cols_removed']
        s_index= pfam_dict['s_index']
        s_ipdb = pfam_dict['s_ipdb']
        preprocess_data = pfam_dict['msa']
        print('Shape of data which gives our predictions: ',np.array(preprocess_data).shape)
        preprocessed_data = []
        for seq_info in preprocess_data:
            preprocessed_data.append([char for char in seq_info[1]])
        print(np.array(preprocessed_data).shape)






	Cannot match Reference sequence to MSA structure
  -->MSA curated by ref-seq loses all sequences
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



sequence indices which match ref seq: 
 [188]
sequences which match ref seq: 
 ['---------------------------------F--GPELVTLDPEEQFTVLSLSA----------G----------R----------P--------K--RPHARRALCLLLGPDFFTDVITIETADHARLQLQLA---'] 

#------------------------------------------------#

first matching indx in alignment:  0
first matching seq: in alignment:  ---------------------------------F--GPELVTLDPEEQFTVLSLSA----------G----------R----------P--------K--RPHARRALCLLLGPDFFTDVITIETADHARLQLQLA---
['F', 'G', 'P', 'E', 'L', 'V', 'T', 'L', 'D', 'P', 'E', 'E', 'Q', 'F', 'T', 'V', 'L', 'S', 'L', 'S', 'A', 'G', 'R', 'P', 'K', 'R', 'P', 'H', 'A', 'R', 'R', 'A', 'L', 'C', 'L', 'L', 'L', 'G', 'P', 'D', 'F', 'F', 'T', 'D', 'V', 'I', 'T', 'I', 'E', 'T', 'A', 'D', 'H', 'A', 'R', 'L', 'Q', 'L', 'Q', 'L', 'A']
['F', 'G', 'P', 'E', 'L', 'V', 'T', 'L', 'D', 'P', 'E', 'E', 'Q', 'F', 'T', 'V', 'L', 'S', 'L', 'S', 'A', 'G', 'R', 'P', 'K', 'R', 'P', 'H', 'A', 'R', 'R', 'A', 'L', 'C', 'L', 'L', 'L', 'G', 'P', 'D', 'F',

SystemExit: 

UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.

In [ ]:
computing_DI = True
if computing_DI:
        # Compute DI scores using Expectation Reflection algorithm
        erdca_inst = erdca.ERDCA(
            preprocessed_data_outfile,
            'PROTEIN',
            s_index = s_index,
            pseudocount = 0.5,
            num_threads = 4,
            seqid = 0.8)

        # Compute average product corrected Frobenius norm of the couplings
        start_time = timeit.default_timer()
        erdca_DI = erdca_inst.compute_sorted_DI()
        run_time = timeit.default_timer() - start_time
        print('ER run time:',run_time)

        for site_pair, score in erdca_DI[:5]:
            print(site_pair, score)

        with open('DI/ER/er_DI_%s.pickle'%(pfam_id), 'wb') as f:
            pickle.dump(erdca_DI, f)
        f.close()
else:
        with open('DI/ER/er_DI_%s.pickle'%(pfam_id), 'rb') as f:
            erdca_DI = pickle.load( f)
        f.close()


In [ ]:
# Print Details of protein PDB structure Info for contact visualizeation
erdca_plotting = True
if erdca_plotting:
    
    # translate DI indices to pdb range:
    pdb_s_index = np.arange(pdb_range[0],pdb_range[1])
    pdb_s_index = np.delete(pdb_s_index,cols_removed)
    print(s_index)
    print(pdb_s_index)
    pdb_range_DI = dict()
    for (indx1,indx2), score in erdca_DI:
            if indx1 in s_index and indx2 in s_index:
                pos1 = pdb_s_index[np.where(s_index==indx1)[0][0]] - pdb_range[0]
                pos2 = pdb_s_index[np.where(s_index==indx2)[0][0]] - pdb_range[0]
                indices = (pos1,pos2)
                pdb_range_DI[indices] = score
    print(erdca_DI[:10])
    pdb_range_DI  = sorted(pdb_range_DI.items(), key = lambda k : k[1], reverse=True)
    print(pdb_range_DI[:10])

    print('Using chain ',pdb_chain)
    print('PDB ID: ', pdb_id)

    
    
    from pydca.contact_visualizer import contact_visualizer
    erdca_visualizer = contact_visualizer.DCAVisualizer('protein', pdb_chain, pdb_id,
    refseq_file = pp_ref_file,
    #---------------- DI For Visualization ----------------#                                                    
    sorted_dca_scores = pdb_range_DI,
    #------------------------------------------------------#        
    linear_dist = 5,
    contact_dist = 10.)

    
    
    
    er_contact_map_data = erdca_visualizer.plot_contact_map()
    plt.show()
    plt.savefig('contact_map_%s.pdf'%pfam_id)
    print(er_contact_map_data.keys())
    print("TP contacts: ",len(er_contact_map_data['tp']))
    print("FP contacts: ",len(er_contact_map_data['fp']))
    print("Missing Contacts: ",len(er_contact_map_data['missing']))
    print("PDB Contacts ",len(er_contact_map_data['pdb']))


    plt.close()
    er_tp_rate_data = erdca_visualizer.plot_true_positive_rates()
    plt.show()
    print(len(er_tp_rate_data))
    print(len(er_tp_rate_data[0]['dca']))
    print(len(er_tp_rate_data[0]['pdb']))
    print(er_tp_rate_data[1])

    plt.savefig('TP_rate_%s.pdf'%pfam_id)
    plt.close()


In [ ]:
# Updating contact map to use pdb-refs pdb_start and pdb_end when getting pp sequence coordinates.
ct,ct_full,n_amino_full,poly_seq = tools.contact_map(pdb,ipdb,cols_removed,s_index)

print('before distance restr on contact map predicitons\ncontact map shape: ',ct.shape)
# Want to only ROC-analyze positions we made predictions for !!!
ct_predict = np.asarray(tools.distance_restr_ct(ct,s_index,make_large = True))
print('Prediction Contact map shape (distance enforced): ',ct_predict.shape)


#--------------------- Generate DI Matrix ---------------------------#
di = np.zeros((n_amino_full,n_amino_full))
for coupling in erdca_DI:
        #print(coupling[1])
        di[coupling[0][0],coupling[0][1]] = coupling[1]
        di[coupling[0][1],coupling[0][0]] = coupling[1]
#--------------------------------------------------------------------#
# Generate DI matrix of our predictions only    
di_predict = np.zeros((len(s_index),len(s_index)))
for coupling in erdca_DI:
        #print(coupling[1])
        if coupling[0][0] in s_index and coupling[0][1] in s_index:
                di_predict[np.where(s_index==coupling[0][0]),np.where(s_index==coupling[0][1])] = coupling[1]
                di_predict[np.where(s_index==coupling[0][1]),np.where(s_index==coupling[0][0])] = coupling[1]



print('\n\n #------------------ Calculating ROC Curve --------------------#')

#----------------- Generate Optimal ROC Curve -----------------------#
# find optimal threshold of distance for both DCA and ER
ct_thres = np.linspace(1.5,10.,18,endpoint=True)
n = ct_thres.shape[0]

auc = np.zeros(n)




for i in range(n):
        #p,tp,fp = tools.roc_curve(ct_distal,di,ct_thres[i])
        p,tp,fp = tools.roc_curve(ct_predict,di_predict,ct_thres[i])
        auc[i] = tp.sum()/tp.shape[0]
i0 = np.argmax(auc)
print('Optimal Distance: ',ct_thres[i0])


# set true positivies, false positives and predictions for optimal distance
p0,tp0,fp0 = tools.roc_curve(ct_predict,di_predict,ct_thres[i0])
print(tp0)
print(fp0)
